In [1]:

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer
from joblib import load

from sqlalchemy import create_engine
import pymssql
from datetime import datetime

from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass

KEY = "PKXP6PHRQXOD9Y7ZQZLR"
SECRET_KEY = "1b1gHrYXJbi3WPXaEbeq72WYdFs9r9ODZ04RvMF7"
numstocks = 200

qt = QuantileTransformer(output_distribution="normal")
pipe = load("files/linear_model_2023-01-20.joblib")

c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator PolynomialFeatures from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
c:\Users\Risto Trajanov\anaconda3\envs\myenv\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator Pipeline from version 1.1.2 when using version 1.0.2. 

## Rank stocks

#### Get features

In [2]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

In [3]:
df = pd.read_sql(
    """
    select ticker, date, bm, mom12m, roeq, mve, famaindustry
    from today
    where price > 5
    """, 
    conn
)
conn.close()


df = df.dropna()
df = df.set_index("ticker")
df = df.sort_values(by="mve")
df = df.iloc[:-500]

features = ["bm", "mom12m", "roeq"]

#### Predict and rank

In [4]:
trans_features = qt.fit_transform(df[features])
trans_features = pd.DataFrame(trans_features, columns=features)
df["predict"] = pipe.predict(trans_features)
df["rnk"] = df.predict.rank(method="first")

for f in features:
    df["t"+f] = qt.fit_transform(df[f].to_numpy().reshape(-1,1))

## Get data from Alpaca

#### Get account equity

In [5]:
trading_client = TradingClient(KEY, SECRET_KEY, paper=True)
account = trading_client.get_account()
equity = float(account.equity)

#### Get tradeable and shortable stocks

In [6]:
assets = trading_client.get_all_assets()

assets = [
    x for x in assets 
    if (x.asset_class[:]=='us_equity') 
    and (x.symbol in df.index) 
    and (x.status[:]=='active')
]
symbols = [x.symbol for x in assets]
tradable = [x.tradable for x in assets]
shortable = [x.shortable for x in assets]

df["tradable"]= pd.Series(tradable, index=symbols)
df["shortable"] = pd.Series(shortable, index=symbols)

#### Get quotes

In [7]:
data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
params = StockLatestQuoteRequest(symbol_or_symbols=df.index.to_list())
quotes = data_client.get_stock_latest_quote(params)

df["ask"] = [quotes[x].ask_price for x in df.index]
df["bid"] = [quotes[x].bid_price for x in df.index]

#### Get positions

In [8]:
positions = trading_client.get_all_positions()
if len(positions) > 0:
    positions = {x.symbol: int(x.qty) for x in positions}
    positions = pd.Series(positions)
    df["current"] = positions
    df["current"] = df.current.fillna(0)
else:
    df["current"] = 0

In [6]:
df = df.sort_values(by="rnk")
short_cutoff = df.rnk.iloc[numstocks-1]
long_cutoff = df.rnk.iloc[-numstocks]

In [8]:
short_cutoff

200.0

In [7]:
long_cutoff

1696.0

## Trade

#### Calculate target positions

In [9]:
long_per_stock = 1.3*equity / numstocks
short_per_stock = 0.3*equity / numstocks

df = df.sort_values(by="rnk")

try:
    short_cutoff = df[df.shortable & (df.bid>0)].rnk.iloc[numstocks-1]
    long_cutoff = df[df.tradable & (df.ask>0)].rnk.iloc[-numstocks]
    df["target"] = np.where(
        df.shortable & (df.bid>0) & (df.rnk<=short_cutoff),
        -short_per_stock/df.bid, 
        0
    )
    df["target"] = np.where(
        df.tradable & (df.ask>0) & (df.rnk>=long_cutoff), 
        long_per_stock/df.ask, 
        df.target
    )
    df["target"] = df.target.astype(int)
except:
    df["target"] = 0

#### Calculate trades

Using a simple but suboptimal protocol: trade to target positions without trying to minimize the number of round trips we might eventually make.

In [10]:
df["trade"] = df.target - df.current

In [9]:
for tick in df.index:
    print(tick)

CVNA
PBT
RVNC
EAR
WISA
MSGM
TUSK
NUWE
ASAN
PTON
HALL
RDFN
KTRA
MCRB
TREE
AXSM
NUZE
KOD
NOTV
ROOT
GRPN
MCG
FATE
NSTG
BIVI
LYFT
MDGL
EGHT
CDLX
RYTM
IHRT
ACHV
BIGC
TISI
LPSN
SFIX
LAW
UPST
GH
OGEN
SMG
AMCX
MARA
IOVA
OPRT
HBI
NYC
WK
AIP
ETNB
FENC
RETA
GDRX
APP
AFRM
APRE
ATOM
MGTX
ASPN
ESTA
WMC
FHTX
AOMR
CABA
TKNO
GHSI
AMPL
RANI
CFLT
ZUO
FTDR
VERA
OSCR
EXFY
TELA
INSM
DGLY
PROK
TMDX
CNCE
TNDM
CDXS
ZETA
NTLA
LUNG
CCCC
MRTX
SYM
VVI
SWTX
RRGB
CELH
PEGA
AIRS
S
RARE
VRNA
FGEN
IMNM
SLDB
CBAY
CPS
SG
AKRO
ARQT
BLUE
ATEC
KALV
BPMC
PVBC
MGNX
RXDX
TVTX
PAY
NTRA
INBS
KA
MRNS
PTGX
ALEC
CDNA
VERI
SPR
HEAR
NCLH
OPNT
BLZE
RIOT
COUP
OCUL
FSLY
BRLT
TSE
APPS
DFFN
ACCD
IPDN
MRSN
GPP
PETQ
EHTH
OLPX
HCAT
TWOU
EB
WEAV
APLS
CUTR
FTHM
OPRX
ZNTL
SHLS
BIG
CTGO
SWI
RMBL
PLRX
HCP
UVE
STOK
VTYX
SGFY
PRVB
TERN
EDIT
JYNT
ATXS
APPN
AURA
TCX
QTRX
ARVN
TSVT
KZR
RLYB
FIVN
APEN
NICK
BAND
ALBO
BTAI
EOLS
PCVX
LMND
ALVR
ESTC
ADEA
ORC
ANNX
INOD
HA
SEMR
TDOC
GLDD
AEYE
VRDN
FCUV
WRBY
ARCT
OLO
OMGA
HRB
UPWK
RRR
BEAM
DLA
QTWO
UFI
VOR
LZ

#### Make trades

In [11]:
for tick in df.index: 
    if df.loc[tick, "trade"]<0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=-df.loc[tick, "trade"],
                side=OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
        except:
            print(f"sell order for {tick} failed")
    elif df.loc[tick, "trade"]>0:
        try:
            market_order_data = MarketOrderRequest(
                symbol=tick,
                qty=df.loc[tick, "trade"],
                side=OrderSide.BUY,
                time_in_force=TimeInForce.DAY
            )
            market_order = trading_client.submit_order(
                order_data=market_order_data
            )
        except:
            print(f"buy order for {tick} failed")



## Save data

In [12]:
today = datetime.today().strftime("%Y-%m-%d")

df["date"] = today

try:
    d = pd.read_csv("files/trade_data.csv", index_col="ticker")
    d = d[d.date != today]
    df = pd.concat((d, df))
    df.to_csv("files/trade_data.csv")
except:
    df.to_csv("files/trade_data.csv")

In [13]:
account = trading_client.get_account()
account = dict(account)
account = pd.DataFrame(pd.Series(account)).T
account["date"] = today

try:
    d = pd.read_csv("files/account.csv")
    d = d[d.date != today]
    account = pd.concat((d, account))
    account.to_csv("files/account.csv")
except:
    account.to_csv("files/account.csv")


C:\Users\keb7\AppData\Local\Temp\ipykernel_41848\1684087778.py:9: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  account = pd.concat((d, account))


In [14]:
positions = trading_client.get_all_positions()
positions = {x.symbol: x.qty for x in positions}
positions = pd.DataFrame(pd.Series(positions))
positions["date"] = today

try:
    d = pd.read_csv("files/positions.csv")
    d = d[d.date != today]
    positions = pd.concat((d, positions))
    positions.to_csv("files/positions.csv")
except:
    positions.to_csv("files/positions.csv")